# Data Augmentation

More data typically increases model performance. So, what do we do if we have small amounts of image training data and cannot collect more? One popular method is data augmentation.

**Data augmentation** is the increase of an existing training dataset's size and diversity without the requirement of manually collecting any new data. The approach generates additional training data from existing examples by augmenting them using random transformations that yield believable-looking images, which exposes the model to more aspects of the data and helps it generalize better.

We train a deep learning model to tune its parameters so it can effectively map a particular input (e.g., an image) to some output (e.g., a label). The goal is to find the sweet spot where the model’s loss is low, which happens when your parameters are tuned in the right way.

Specifically, data augmentation is acquired by performing a series of random preprocessing transformations to existing data such as horizontal and vertical flipping, skewing, cropping, shearing, zooming in and out, and rotating. Collectively, augmented data is able to simulate a variety of subtly different data points as opposed to just duplicating the same data. The subtle differences of the augmented images should (hopefully) be enough to help train a more robust model.

Data augmentation can also mitigate overfitting. Overfitting generally occurs when there are a small number of training examples. By generating additional training data from existing examples, we may mitigate overfitting.

General resource:

https://www.charterglobal.com/image-data-augmentation-and-ai/

# Import **tensorflow** Library

In [ ]:
import tensorflow as tf

# GPU Hardware Accelerator

To vastly speed up processing, we can use the GPU available from the Google Colab cloud service. Colab provides a free Tesla K80 GPU of about 12 GB. It’s very easy to enable the GPU in a Colab notebook:

1.	click **Runtime** in the top left menu
2.	click **Change runtime** type from the drop-down menu
3.	choose **GPU** from the Hardware accelerator drop-down menu
4.	click **SAVE**

Verify that GPU is active:

In [ ]:
tf.__version__, tf.test.gpu_device_name()

If '/device:GPU:0' is displayed, the GPU is active. If '..' is displayed, the regular CPU is active.

# Work with Flowers Data

Import libraries:

In [ ]:
import numpy as np
import PIL.Image

Download data with **tf.keras.utils.get_file** utility:

In [ ]:
import pathlib

url1 = 'https://storage.googleapis.com/download.tensorflow.org/'
url2 = 'example_images/flower_photos.tgz'
dataset_url = url1 + url2

data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='flower_photos', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

## Split Data

Load images off disk using **tf.keras.preprocessing.image_dataset_from_directory** utility into train and test sets.

Set parameters:

In [ ]:
BATCH_SIZE = 32
img_height = 180
img_width = 180

Batch size is set to 32. Since images have different shapes, images are resized to 180 x 180 for model consumption.

### Create Train and Test Sets

The **tf.keras.preprocessing.image_dataset_from_directory** utiltity reads the data from a directory. It then splits, seeds, resizes, and batches the data.

Create the train set:

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.19,
  subset='training',
  seed=0,
  image_size=(img_height, img_width),
  batch_size=BATCH_SIZE)

Set **validation_split=0.19** and **subset='training'**, to get 81% train data.

Create the test set:

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.19,
  subset='validation',
  seed=0,
  image_size=(img_height, img_width),
  batch_size=BATCH_SIZE)

Set **validation_split=0.19** and **subset='validation'**, to get 19% test data.

## Inspect Train Set

Take an example batch and display image shape and label:

In [ ]:
for images, labels in train_ds.take(1):
  print ('image shape:', images.shape)
  print ('labels:', labels.numpy())
  print ('number of labels in a batch:', len(labels))

As expected, we have 32 180 x 180 images in the first batch. We also have 32 corresponding labels.

## Get Number of Classes

Display number of labels:

In [ ]:
class_names = train_ds.class_names
num = len(class_names)
num

## Create Scale Function

Create a function to scale images:

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.0
  return image, label

## Build the Input Pipeline

Scale train and test sets. Shuffle train data, cache, ahd prefetch train and test data.

In [ ]:
SHUFFLE_SIZE = 100

train_fds = train_ds.map(scale).shuffle(SHUFFLE_SIZE).\
            cache().prefetch(1)
test_fds = test_ds.map(scale).cache().prefetch(1)

<!-- Use AUTOTUNE: -->

# Apply Data Augmentation with Preprocessing Layers

Apply data augmentation with experimental Keras Preprocessing Layers. We include the layers inside a model like other layers. Let's augment images by randomly flipping horizonatally, rotating, zooming, and translating.

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing\
  import RandomFlip
from tensorflow.keras.layers.experimental.preprocessing\
  import RandomRotation
from tensorflow.keras.layers.experimental.preprocessing\
  import RandomZoom
from tensorflow.keras.layers.experimental.preprocessing\
  import RandomTranslation


data_augmentation = tf.keras.Sequential(
  [
   RandomFlip('horizontal'),
   RandomRotation(0.1),
   RandomZoom(0.1),
   RandomTranslation(height_factor=0.2, width_factor=0.2)
  ]
)

Resources:

https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing

https://www.tensorflow.org/tutorials/images/data_augmentation

## Display an Augmented Image

Here is what happens when applying data augmentation to the same image several times:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, _ in train_fds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0])
    plt.axis('off')

## Clear Models and Generate Seed

Clear all previous models and generate a seed for reproducibility:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

## Import Libraries

Import requisite libraries to build the model:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Conv2D, MaxPooling2D

## Build Model

Build a multi-layered CNN:

In [ ]:
model = tf.keras.Sequential([
  data_augmentation,
  Conv2D(32, 3, activation='relu'),
  MaxPooling2D(),
  Conv2D(32, 3, activation='relu'),
  MaxPooling2D(),
  Conv2D(32, 3, activation='relu'),
  MaxPooling2D(),
  Flatten(),
  Dense(128, activation='relu'),
  Dropout(0.5),
  Dense(num)
])

## Compile

Compile with SparseCategoricalCrossentropy(from_logits=True):

In [ ]:
model.compile(
  optimizer='adam',
  loss=SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

## Train

Train model for three epochs:

In [ ]:
history = model.fit(
    train_fds,
    validation_data=test_fds,
    epochs=10)

# Apply Data Augmentation Directly on Images

Instead of creating a training model with an augmentation layer, we can perform transformations directly on images.

General resource:

https://medium.com/mlait/image-data-augmentation-image-processing-in-tensorflow-part-2-b77237256df0

TensorFlow resource:

https://towardsdatascience.com/tensorflow-image-augmentation-on-gpu-bf0eaac4c967

Grab some images from the training set we created earlier in the notebook:

In [ ]:
for batch_images, _ in train_fds.take(1):
  print ('image shape:', batch_images.shape)

Since a training set of images was already created earlier in the notebook, just grab the first batch. Now, we have a batch of 32 images with which to play.

## Set Index

Since batch size is 32, we can set the value from 0 to 31.

In [ ]:
indx = 0
indx

We set the index to grab the first image from the batch. Change the index value (between 0 and 31) to display different images.

Image:

In [ ]:
our_image = batch_images[indx]

## Show an Example

Display the image:

In [ ]:
plt.imshow(our_image)
plt.axis('off')
plt.grid(b=None)

## Create Functions to Show Images

Show original and modified image:

In [ ]:
def show(original_img, trans_img):
  f = plt.figure(figsize=(6, 6))
  f.add_subplot(1,2,1)
  plt.imshow(original_img)
  plt.axis('off')
  f.add_subplot(1,2,2)
  plt.imshow(trans_img)
  plt.axis('off')
  plt.show(block=True)

Show several transformations of an image:

In [ ]:
def show_images(img, indx, trans, p1=None, p2=None, b=False):
  plt.figure(figsize=(10, 10))
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    if not b:
      new_img = trans(img[indx])
    elif p2==None:
      new_img = trans(img[indx], p1)
      new_img = np.clip(new_img, 0, 1)
    else:
      new_img = trans(img[indx], p1, p2)
      new_img = np.clip(new_img, 0, 1)
    plt.imshow(new_img)
    plt.axis('off')

## Demonstrate Examples

Let's demonstrate how to augment directly on images.

Resource:

https://www.tensorflow.org/api_docs/python/tf/image/

## Crop an Image

To **crop** an image, remove or adjust its outside edges.

Crop:


In [ ]:
new_image = tf.image.random_crop(our_image, [120, 120, 3])
new_image.shape

Show original and modified image:

In [ ]:
show(our_image, new_image)

Since the transformation is random, the image is cropped randomly:

In [ ]:
show_images(batch_images, indx, tf.image.random_crop,
            [120, 120, 3], b=True)

Centrally crop:

In [ ]:
new_image = tf.image.central_crop(our_image, 0.5)
print (new_image.shape)
show(our_image, new_image)

Crop to a bounding box:

In [ ]:
new_image = tf.image.crop_to_bounding_box(
    our_image, 10, 10, 120, 120)
print (new_image.shape)
show(our_image, new_image)


## Randomly Flip Image Left to Right

Since the transformation is random, the image isn't always flipped:

In [ ]:
show_images(batch_images, indx, tf.image.random_flip_left_right)

## Randomly Flip Image Up to Down

Since the transformation is random, the image isn't always flipped:

In [ ]:
show_images(batch_images, indx, tf.image.random_flip_up_down)

## Flip Image Up to Down

The image is always flipped:

In [ ]:
show(our_image, tf.image.flip_up_down(our_image))

## Rotate Image 90 Degrees

Each **k** value rotates the image 90 degrees.

Rotate 90 degrees:

In [ ]:
show(our_image, tf.image.rot90(our_image, k=1))

Rotate 180 degrees:

In [ ]:
show(our_image, tf.image.rot90(our_image, k=2))

Rotate 270 degrees:

In [ ]:
show(our_image, tf.image.rot90(our_image, k=3))

Rotate 360 degrees:

In [ ]:
show(our_image, tf.image.rot90(our_image, k=4))

We're back where we started!

## Adjust Gamma

**Gamma encoding** can be thought of as the intensity of an image. It is used to optimize the usage of bits when encoding an image by taking advantage of the non-linear manner that humans perceive light and color. The human perception of brightness (lightness) under common illumination conditions (neither pitch black nor blindingly bright) follows an approximate power function with greater sensitivity to relative differences between darker tones than between lighter tones.

Resource:

https://en.wikipedia.org/wiki/Gamma_correction

Adjust with gamma encoding:

In [ ]:
new_image = tf.image.adjust_gamma(
    our_image, gamma=0.75, gain=1.5)
new_image = np.clip(new_image, 0, 1)
show(our_image, new_image)

Adjust gamma for brightness. For gamma less than 1, image is brighter. For gamma greater than 1, image is darker. Intensity is controlled by gain. Apply NumPy clip function to ensure that pixel values are still in between [0, 1].

## Adjust Contrast

**Contrast** in image processing is the difference in luminance (or color) that makes an object distinguishable. It is determined by the difference in the color and brightness of the object and other objects within the same field of view.

Fixed contrast:

In [ ]:
new_image = tf.image.adjust_contrast(
    our_image, contrast_factor=1.8)
new_image = np.clip(new_image, 0, 1)
show(our_image, new_image)

Adjust contrast_factor for more or less luminance.

Random contrast:

In [ ]:
show_images(batch_images, indx, tf.image.random_contrast,
            0.75, 2.9, b=True)

## Adjust Brightness

Fixed brightness:

In [ ]:
new_image = tf.image.adjust_brightness(our_image, .25)
new_image = np.clip(new_image, 0, 1)
show(our_image, new_image)

Random brightness:

In [ ]:
show_images(batch_images, indx, tf.image.random_brightness,
            0.25, b=True)

## Adjust Saturation

**Saturation** in image processing is the depth or intensity of color present within an image. The more saturated an image, the more colorful and vibrant it appears. Less color saturation makes an image appear subdued or muted.

Fixed saturation:

In [ ]:
show(our_image, tf.image.adjust_saturation(our_image, 3.0))

Random saturation:

In [ ]:
show_images(batch_images, indx, tf.image.random_saturation,
            0.3, 3.5, b=True)

## Adjust Hue

**Hue** is the main indication of a RGB color. It is the value that tells us whether an object is red, green or blue. In contrast, saturation is the perceived intensity. So saturation is how colorful the object looks, while hue is the actual color.

Fixed hue:

In [ ]:
show(our_image, tf.image.adjust_hue(our_image, 0.2))

Random hue:

In [ ]:
show_images(batch_images, indx, tf.image.random_hue, 0.2, b=True)

## Create an Augmentation Function

Randomly flip images from left to right apply a saturation operation to images:

In [ ]:
def augment(image, label):
  img = tf.image.random_flip_left_right(image)
  final_image = tf.image.random_saturation(img, 0, 2)
  return (final_image, label)

## Display an Augmented Image

Here is what happens when the **augment** function is applied to the same image several times:

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  image, _ = augment(our_image, labels[0])
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image)
  plt.axis('off')

## Build the Input Pipeline

Build a pipeline with train and test data:

In [ ]:
SHUFFLE_SIZE = 10000

train1 = train_ds.map(scale, num_parallel_calls=4)
train2 = train1.map(augment, num_parallel_calls=4)
train_da = train2.shuffle(SHUFFLE_SIZE).cache().prefetch(1)
test_da = test_ds.map(scale).cache().prefetch(1)

Use **num_parallel** parameter for better performance.

## Clear Previous Models and Generate Seed

Clear previous models and seed:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

## Create Model

Create a multi-layer CNN:

In [ ]:
model = tf.keras.Sequential([
  Conv2D(32, 3, activation='relu',
         input_shape=[180, 180, 3]),
  MaxPooling2D(),
  Conv2D(32, 3, activation='relu'),
  MaxPooling2D(),
  Conv2D(32, 3, activation='relu'),
  MaxPooling2D(),
  Flatten(),
  Dense(128, activation='relu'),
  Dropout(0.5),
  Dense(num)
])

## Compile

Compile with SparseCategoricalCrossentropy(from_logits=True):

In [ ]:
model.compile(
  optimizer='adam',
  loss=SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

## Train

Train the model:

In [ ]:
history = model.fit(
    train_da,
    validation_data=test_da,
    epochs=5)

# Data Augmentation with ImageGenerator

So far, we have augmented images in two ways. We utilized Keras preprocessing layers and applied TensorFlow operations directly on images to augment. Both methods build a tf.data pipeline to model the augmented dataset.

An alternative is to use the ImageGenerator class. The ImageGenerator class makes it easy to load images from disk and augment them in various ways.
We can shift, rotate, rescale, flip horizontally or vertically, shear or apply transformation functions to images. Although ImageGenerator is very convenient for simple projects, building a tf.data pipeline is more conducive to complex projects because it can read images in parallel from any source (not just a local disk), manipulate the dataset in any manner, and preprocessing functions based on tf.image operations can be used in the tf.data pipeline and in the model deployed to production.


Resource:

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

## Use ImageDataGenerator to Process Flowers Data

Load data directly from the flowers directory into train and test splits by utilizing dictionaries. The first dictionary provides scaling and splitting information to the ImageDataGenerator class. The second dictionary provides target and batch information to the class.

Import the appropriate library:

In [ ]:
from tensorflow.keras.preprocessing.image\
  import ImageDataGenerator

Create a dictionary to scale and split data:

In [ ]:
datagen_kwargs = dict(rescale=1./255, validation_split=.19)

Split is 81/19.

Create a dictionary to resize, batch, and interpolate:

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = (180, 180)

dataflow_kwargs = dict(target_size=IMAGE_SIZE,
                       batch_size=BATCH_SIZE,
                       interpolation='bilinear')

### Create Validation Data

Create the test set:

In [ ]:
valid_datagen = tf.keras.preprocessing.image.\
  ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset='validation', shuffle=False,
    **dataflow_kwargs)

### Create Train Data

Create the train set:

In [ ]:
train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset='training', shuffle=True,
    **dataflow_kwargs)

### Clear Models and Generate Seed

Clear previous models and generate a seed:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

### Create Model

Create a multi-layer CNN:

In [ ]:
model = Sequential([
  Conv2D(filters=32, kernel_size=(5, 5), activation = 'relu',
         input_shape=(180, 180, 3)),
  MaxPooling2D(2),
  Conv2D(64, (5, 5), activation='relu'),
  MaxPooling2D(2),
  Flatten(),
  Dense(64, activation='relu'),
  Dense(5, activation='softmax')
])

### Compile

Compile with categorical_crossentropy:

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train

Train for one epoch:

In [ ]:
history = model.fit(train_generator, batch_size=BATCH_SIZE,
                    epochs=5, validation_data=valid_generator,
                    verbose=1)

## Use ImageDataGenerator to Augment Train Data

Augment with **tf.keras.preprocessing.image.ImageDataGenerator**:

In [ ]:
aug_train_datagen = tf.keras.preprocessing.\
  image.ImageDataGenerator(
      rotation_range=40, horizontal_flip=True,
      width_shift_range=0.2, height_shift_range=0.2,
      shear_range=0.2, zoom_range=0.2,
      **datagen_kwargs)

Create train set with data augmentation:

In [ ]:
aug_train_generator = aug_train_datagen.flow_from_directory(
    data_dir, subset='training', shuffle=True, **dataflow_kwargs)

Only augment train data!

Clear model and generate seed:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

Recompile model:

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Train:

In [ ]:
history = model.fit(aug_train_generator, batch_size=BATCH_SIZE,
                    epochs=5, validation_data=valid_generator,
                    verbose=1)

### Inspect the Data

Grab a batch from the original training set:

In [ ]:
data_list = []
batch_index, end_index = 0, 1
while batch_index <= train_generator.batch_index:
  if batch_index < end_index:
    data = train_generator.next()
    data_list.append(data[0])
    batch_index = batch_index + 1
  else: break
original = np.asarray(data_list)


Inspect the shape:

In [ ]:
original[0].shape

Grab a batch from the augmented training set:

In [ ]:
data_list = []
batch_index, end_index = 0, 1
while batch_index <= aug_train_generator.batch_index:
  if batch_index < end_index:
    data = aug_train_generator.next()
    data_list.append(data[0])
    batch_index = batch_index + 1
  else: break
augmented = np.asarray(data_list)

Inspect the shape:

In [ ]:
augmented[0].shape

Visualize a training image:

In [ ]:
train_image = original[0][0]

plt.imshow(train_image)
plt.axis('off')
plt.grid(b=None)

Visualize several training images:

In [ ]:
plt.figure(figsize=(10, 10))
for images in original:
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.axis('off')

Visualize an augmented image:

In [ ]:
aug_train_image = augmented[0][0]

plt.imshow(aug_train_image)
plt.axis('off')
plt.grid(b=None)

Visualize several augmented images:

In [ ]:
plt.figure(figsize=(10, 10))
for images in augmented:
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.axis('off')

The original training images look like normal flower images. However, we see that the augmented images are definitely transformed.